In [3]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

In [4]:
population = pd.read_csv('data/1970-2024 인구데이터.csv', encoding='EUC-KR')
population

,시도별,성별,연령별,시점,추계인구
0,전국,계,계,1970 년,32240827.0
1,전국,계,계,1971 년,32882704.0
2,전국,계,계,1972 년,33505406.0
3,전국,계,계,1973 년,34103149.0
4,전국,계,계,1974 년,34692266.0
...,...,...,...,...,...
58585,제주특별자치도,여자,100세 이상,2020 년,129.0
58586,제주특별자치도,여자,100세 이상,2021 년,145.0
58587,제주특별자치도,여자,100세 이상,2022 년,182.0
58588,제주특별자치도,여자,100세 이상,2023 년,270.0


In [5]:
# 데이터를 보면 데이터와 80-100세까지 나눠진 데이터의 합이 80세 이상의 row에 나타나있음.
# 연도, 연도별 전체인구 수(연령 상관 X), 각 연도별 65세 이상 인구수 = 고령인구 비율 

# 각 연도별 65세 이상 인구 구하기
population['시도별'] = population['시도별'].str.strip()
age_list = ["65 - 69세", "70 - 74세", "75 - 79세", "80세이상"]


test = population.query('시도별 != "전국" and 성별 == "계" and 연령별 in @age_list')
test
old_year_sum = (
    test.groupby(['시점', '시도별'])['추계인구']
    .sum()
    .reset_index()
    .rename(columns={'추계인구': '노년층_인구'})
)
population = population.merge(old_year_sum, on=['시점', '시도별'], how='left')

population.rename(columns={
    '추계인구': '지역별_총_인구수'
}, inplace=True)


# 시/도 이름 매핑 딕셔너리
replace_dict = {
    '서울특별시': '서울',
    '부산광역시': '부산',
    '대구광역시': '대구',
    '인천광역시': '인천',
    '광주광역시': '광주',
    '대전광역시': '대전',
    '울산광역시': '울산',
    '세종특별자치시': '세종시',
    '경기도': '경기',
    '강원도': '강원',
    '충청북도': '충북',
    '충청남도': '충남',
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '제주특별자치도': '제주'
}

# 주소 열 변환
population['시도별'] = population['시도별'].replace(replace_dict, regex=True)

population.query('시도별 != "전국" and 성별 == "계" and 연령별 == "계"')

,시도별,성별,연령별,시점,지역별_총_인구수,노년층_인구
3345,서울,계,계,1970 년,5685932.0,95538.0
3346,서울,계,계,1971 년,5940442.0,106698.0
3347,서울,계,계,1972 년,6206625.0,110568.0
3348,서울,계,계,1973 년,6470497.0,119326.0
3349,서울,계,계,1974 년,6737832.0,129836.0
...,...,...,...,...,...,...
55295,제주,계,계,2020 년,668699.0,101167.0
55296,제주,계,계,2021 년,672079.0,106173.0
55297,제주,계,계,2022 년,674523.0,110855.0
55298,제주,계,계,2023 년,676557.0,116121.0


In [9]:
# 조건 적용: 시도별이 전국아니고, 성별이 계, 연령별이 노년층인 데이터만 추출
filtered = population.query('시도별 != "전국" and 성별 == "계" and 연령별 == "계"')

# 해당 조건 반영된 데이터만 CSV로 저장
filtered.to_csv('시도_연도별_노령_인구데이터.csv', encoding='utf-8-sig', index=False)

eledly_health = population.query('시도별 != "전국" and 성별 == "계" and 연령별 == "계" and 시점 == "2024년"')
eledly_health.to_csv('시_연도별_노령_인구데이터.csv', encoding='utf-8-sig', index=False)